In [1]:
import torch
import os
import models
from charge_trainer import ChargeTrainer
from ocpmodels.common import logger
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import setup_logging
from DeepDFT import densitymodel
from chg_utils import ProbeGraphAdder
setup_logging()

import matplotlib.pyplot as plt
from torch_geometric.data import Batch

In [2]:
if torch.cuda.is_available():
    print("True")
else:
    print("False")
    torch.set_num_threads(8)

True


In [3]:
task = {
    'dataset': 'lmdb',
    'description': 'Initial test of training on charges',
    'type': 'regression',
    'metric': ['charge_mse', 'charge_mae'],
    'primary_metric': 'charge_mae',
    'labels': ['charge_vals'],
}

In [4]:
'''
The atom_model_config and probe_model_config should inherit most keywords
from OCP models. The exception is specifications for the number of
interactions and the number of channels. These hyperparameters are needed
in the wrapper model as well, so they are specified outside of the
individual configurations.
'''

model = {
    'name': 'charge_model',
    'num_interactions': 5,
    'atom_channels': 256,
    'probe_channels': 256,
    
    'atom_model_config': {
        'name': 'schnet_charge',
        'num_filters':64,
        'num_gaussians':32,
        'cutoff':5,
    },
    
    'probe_model_config': {
        'name': 'schnet_charge',
        'num_filters':64,
        'num_gaussians':32,
        'cutoff':5,
    },
}

In [5]:
optimizer = {
    'optimizer': 'Adam',
    'batch_size': 10,
    'eval_batch_size': 10,
    'num_workers': 1,
    'lr_initial': 5e-4,
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.96,
    'patience': 1,
    'max_epochs': 1000,
}

In [6]:
dataset = [
{'src': '../chg/100/train', 'normalize_labels': False}, # train set 
{'src': '../chg/100/val'}, # val set (optional)
# {'src': train_src} # test set (optional - writes predictions to disk)
]

In [7]:
trainer_config = {
    'trainer': 'charge',
    'identifier': 'A Good Run',
    'is_debug': False,
    'run_dir': './runs/',
    'print_every': 1,
    'seed': 2,
    'logger': 'wandb',
    'local_rank': 0,
    'amp': True,
    
    'cutoff': 5,
    'train_probes': 500,
    'val_probes': 1000,
    'test_probes': 1000,
}

In [8]:
trainer = registry.get_trainer_class(
    trainer_config['trainer'])(task = task,
                               model = model,
                               dataset = dataset,
                               optimizer = optimizer,
                               **trainer_config)

amp: true
cmd:
  checkpoint_dir: ./runs/checkpoints/2022-07-06-15-02-24-A Good Run
  commit: 7d5240c
  identifier: A Good Run
  logs_dir: ./runs/logs/wandb/2022-07-06-15-02-24-A Good Run
  print_every: 1
  results_dir: ./runs/results/2022-07-06-15-02-24-A Good Run
  seed: 2
  timestamp_id: 2022-07-06-15-02-24-A Good Run
dataset:
  normalize_labels: false
  src: ../chg/100/train
gpus: 1
logger: wandb
model: charge_model
model_attributes:
  atom_channels: 256
  atom_model_config:
    cutoff: 5
    name: schnet_charge
    num_filters: 64
    num_gaussians: 32
  num_interactions: 5
  probe_channels: 256
  probe_model_config:
    cutoff: 5
    name: schnet_charge
    num_filters: 64
    num_gaussians: 32
noddp: false
optim:
  batch_size: 10
  eval_batch_size: 10
  factor: 0.96
  lr_initial: 0.0005
  max_epochs: 1000
  mode: min
  num_workers: 1
  optimizer: Adam
  patience: 1
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: lmdb
  description: Initial test of training on charges
  

2022-07-06 15:01:52 (ERROR): Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emsunshine. Use `wandb login --relogin` to force relogin


2022-07-06 15:01:55 (INFO): Loading dataset: lmdb
2022-07-06 15:01:55 (INFO): Loading model: charge_model
2022-07-06 15:01:55 (INFO): Loaded ChargeModel with 1234179 parameters.


In [9]:
trainer.train()

charge_mse: 4.24e-01, charge_mae: 5.58e-01, loss: 1.12e+00, lr: 5.00e-04, epoch: 1.11e-01, step: 1.00e+00
charge_mse: 4.73e-01, charge_mae: 5.60e-01, loss: 5.60e-01, lr: 5.00e-04, epoch: 2.22e-01, step: 2.00e+00
charge_mse: 2.24e-01, charge_mae: 3.60e-01, loss: 3.60e-01, lr: 5.00e-04, epoch: 3.33e-01, step: 3.00e+00
charge_mse: 1.70e-01, charge_mae: 3.17e-01, loss: 3.17e-01, lr: 5.00e-04, epoch: 4.44e-01, step: 4.00e+00
charge_mse: 1.34e-01, charge_mae: 1.34e-01, loss: 1.34e-01, lr: 5.00e-04, epoch: 5.56e-01, step: 5.00e+00
charge_mse: 4.25e-01, charge_mae: 4.27e-01, loss: 4.27e-01, lr: 5.00e-04, epoch: 6.67e-01, step: 6.00e+00
charge_mse: 3.01e-01, charge_mae: 4.40e-01, loss: 4.40e-01, lr: 5.00e-04, epoch: 7.78e-01, step: 7.00e+00
charge_mse: 2.42e-01, charge_mae: 2.64e-01, loss: 2.64e-01, lr: 5.00e-04, epoch: 8.89e-01, step: 8.00e+00
charge_mse: 3.33e-02, charge_mae: 1.19e-01, loss: 1.19e-01, lr: 5.00e-04, epoch: 1.00e+00, step: 9.00e+00
2022-07-06 15:07:58 (INFO): Evaluating on val.

device 0: 100%|██████████| 1/1 [00:37<00:00, 37.12s/it]

2022-07-06 15:08:35 (INFO): charge_mse: 0.2351, charge_mae: 0.2450, loss: 0.2450, epoch: 1.0000


charge_mse: 2.40e-01, charge_mae: 2.52e-01, loss: 2.52e-01, lr: 5.00e-04, epoch: 1.11e+00, step: 1.00e+01
charge_mse: 1.12e-01, charge_mae: 2.38e-01, loss: 2.38e-01, lr: 5.00e-04, epoch: 1.22e+00, step: 1.10e+01
charge_mse: 1.61e-01, charge_mae: 1.35e-01, loss: 1.35e-01, lr: 5.00e-04, epoch: 1.33e+00, step: 1.20e+01
charge_mse: 1.41e-01, charge_mae: 1.73e-01, loss: 1.73e-01, lr: 5.00e-04, epoch: 1.44e+00, step: 1.30e+01
charge_mse: 1.60e-01, charge_mae: 2.23e-01, loss: 2.23e-01, lr: 5.00e-04, epoch: 1.56e+00, step: 1.40e+01
charge_mse: 8.46e-02, charge_mae: 1.49e-01, loss: 1.49e-01, lr: 5.00e-04, epoch: 1.67e+00, step: 1.50e+01
charge_mse: 8.72e-02, charge_mae: 1.05e-01, loss: 1.05e-01, lr: 5.00e-04, epoch: 1.78e+00, step: 1.60e+01
charge_mse: 1.20e-01, charge_mae: 1.77e-01, loss: 1.77e-01, lr: 5.00e-04, epoch: 1.89e+00, step: 1.70e+01
charge_mse: 6.93e-02, charge_mae: 1.45e-01, loss: 1.45e-01, lr: 5.00e-04, epoch: 2.00e+00, step: 1.80e+01
2022-07-06 15:09:56 (INFO): Evaluating on val.

device 0: 100%|██████████| 1/1 [00:11<00:00, 11.73s/it]

...

charge_mse: 4.26e-04, charge_mae: 6.14e-03, loss: 6.14e-03, lr: 4.88e-05, epoch: 1.51e+02, step: 1.36e+03
charge_mse: 4.26e-04, charge_mae: 5.84e-03, loss: 5.84e-03, lr: 4.88e-05, epoch: 1.51e+02, step: 1.36e+03
charge_mse: 1.93e-04, charge_mae: 5.64e-03, loss: 5.64e-03, lr: 4.88e-05, epoch: 1.51e+02, step: 1.36e+03
charge_mse: 1.73e-04, charge_mae: 5.87e-03, loss: 5.87e-03, lr: 4.88e-05, epoch: 1.51e+02, step: 1.36e+03
charge_mse: 1.74e-04, charge_mae: 6.24e-03, loss: 6.24e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.36e+03
charge_mse: 3.63e-04, charge_mae: 5.84e-03, loss: 5.84e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.36e+03
charge_mse: 1.11e-04, charge_mae: 4.52e-03, loss: 4.52e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 6.48e-04, charge_mae: 7.04e-03, loss: 7.04e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 1.29e-03, charge_mae: 7.76e-03, loss: 7.76e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
2022-07-06 19:09:43 (INFO): Evaluating on val.

device 0: 100%|██████████| 1/1 [00:11<00:00, 11.73s/it]

2022-07-06 19:09:55 (INFO): charge_mse: 0.0064, charge_mae: 0.0106, loss: 0.0106, epoch: 152.0000


charge_mse: 4.59e-04, charge_mae: 7.10e-03, loss: 7.10e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 7.55e-04, charge_mae: 5.76e-03, loss: 5.76e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 1.77e-04, charge_mae: 6.11e-03, loss: 6.11e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 1.55e-04, charge_mae: 6.00e-03, loss: 6.00e-03, lr: 4.88e-05, epoch: 1.52e+02, step: 1.37e+03
charge_mse: 1.33e-04, charge_mae: 4.37e-03, loss: 4.37e-03, lr: 4.88e-05, epoch: 1.53e+02, step: 1.37e+03
charge_mse: 5.71e-04, charge_mae: 6.29e-03, loss: 6.29e-03, lr: 4.88e-05, epoch: 1.53e+02, step: 1.37e+03
charge_mse: 2.82e-04, charge_mae: 6.78e-03, loss: 6.78e-03, lr: 4.88e-05, epoch: 1.53e+02, step: 1.38e+03


KeyboardInterrupt: 

In [ ]:
model = trainer.model.module
loader = iter(trainer.train_loader)

In [ ]:
#torch.cuda.empty_cache()
batch = next(loader)
for subbatch in batch:
    subbatch.probe_data = Batch.from_data_list(subbatch.probe_data)

In [ ]:
pred = model(batch[0].to('cuda'))
true = batch[0].probe_data.target

In [ ]:
true = true.detach().cpu()#.reshape(10000)
pred = pred.detach().cpu()#.reshape(10000)

plt.scatter(true, pred, 
            color='red', 
            s=1,
            label='Predictions')

lb = torch.min(true)
ub = torch.max(true)
plt.plot([lb, ub], [lb,ub], label='Parity line')
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.xlabel('True label')
plt.ylabel('Predicted label')
plt.legend()
plt.gcf().set_dpi(200)
plt.show()

In [ ]:
err = torch.mean(torch.abs(pred - true))
print(err.item())

In [ ]:
print(torch.min(pred).item())
print(torch.max(pred).item())
print(torch.mean(pred).item())
print(torch.std(pred).item())

In [ ]:
print(torch.min(true).item())
print(torch.max(true).item())
print(torch.mean(true).item())
print(torch.std(true).item())